In [1]:
# using %matplotlib inline to immediatly draw plot after run
%matplotlib inline 

# imports
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import dotenv
import datetime
import os
import pickle
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FunctionTransformer, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from abc import ABC, abstractmethod



"""prepare environment variable"""
project_dir = os.path.join(os.path.abspath(''), os.pardir)
dotenv_path = os.path.join(project_dir, '.env')
dotenv.load_dotenv(dotenv_path)

True

# Get Data

In [2]:
def check_path_existance(path):
    return os.path.exists(path)

dataset_folder = os.path.join(project_dir, os.getenv('RAW_FOLDER'), 'house-prices-advanced-regression-techniques')
train_df = None
test_df = None

if check_path_existance(dataset_folder):   
    train_df = pd.read_csv(os.path.join(dataset_folder, 'train.csv'))
    test_df = pd.read_csv(os.path.join(dataset_folder, 'test.csv'))

In [3]:
train_df.loc[:, ('MSSubClass')] = train_df.loc[:, ('MSSubClass')].astype(str)
test_df.loc[:, ('MSSubClass')] = test_df.loc[:, ('MSSubClass')].astype(str)

# numerical_columns = train_df.select_dtypes(include=['int64', 'float64']).columns
# categorical_columns = train_df.select_dtypes(include=['object']).columns

# Data Preprocessing Pipeline

In [9]:
class DataLoader:
    @staticmethod
    def load_data(path):
        if os.path.exists(path):
            return pd.read_csv(path)
        return None
# data
train_data_path = os.path.join(project_dir, os.getenv('RAW_FOLDER'), 'house-prices-advanced-regression-techniques', 'train.csv')
test_data_path = os.path.join(project_dir, os.getenv('RAW_FOLDER'), 'house-prices-advanced-regression-techniques', 'test.csv')

# print(DataLoader.load_data(train_data_path).shape)
# print(DataLoader.load_data(test_data_path).shape)

In [5]:
class BaseDataPreprocessor(ABC):
    def __init__(self, X: pd.DataFrame):
        self.X = X
    
    @abstractmethod
    def clean_data(self):
        pass

    @abstractmethod
    def transform_data(self):
        pass

    @abstractmethod
    def scale_data(self):
        pass
    
    @abstractmethod
    def select_column_data(self):
        pass

In [46]:
# for data preprocessing
class DataPreprocessor:
    def __init__(self):
        print("DataPreprocessor is succesfully instantiated")
        
    def preprocess_data(self, X: pd.DataFrame, dataset_type: str):
        self.__X = X.copy()
        self.dataset_type = dataset_type
        self.__X = self.clean_data(self.__X) # cleaning data
        # feature engineering
        self.__X = self.select_column_data(self.__X) # feature selection
        self.__X = self.transform_data(self.__X) # transform data
        self.__X = self.scale_data(self.__X, "minmax") # scale data
        return self.__X.copy()

    def clean_data(self, X):
        X.drop(columns=['Id'], inplace=True) # drop irrelavant columns
        X.loc[:, ('MSSubClass')] = X.loc[:, ('MSSubClass')].astype(str) # fix incorrect data type
        
        
        if self.dataset_type == "train":
            ### Clean numerical columns
            numerical_columns = X.select_dtypes(exclude=['object']).columns
            imputer = SimpleImputer(strategy='mean')
            X.loc[:, numerical_columns] = imputer.fit_transform(X.loc[:, numerical_columns]) # impute missing values with mean
            pickle.dump(imputer, open(os.path.join(project_dir, os.getenv('NUMERICAL_IMPUTER_PATH')), 'wb')) # save the imputer


            ### Clean categorical columns
            categorical_columns = X.select_dtypes(include=['object']).columns
            categorical_columns_missing_on_purpose = [
                'Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 
                'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 
                'GarageType', 'GarageFinish', 'GarageQual','GarageCond', 
                'PoolQC', 'Fence', 'MiscFeature'
            ]
            imputer = SimpleImputer(strategy='constant', fill_value='Missing')
            X.loc[:, categorical_columns_missing_on_purpose] = imputer.fit_transform(X.loc[:, categorical_columns_missing_on_purpose]) # impute missing categorical value on purpose

            imputer = SimpleImputer(strategy='most_frequent')
            X.loc[:, categorical_columns] = imputer.fit_transform(X.loc[:, categorical_columns]) # impute missing categorical value not on purpose
            pickle.dump(imputer, open(os.path.join(project_dir, os.getenv('CATEGORICAL_IMPUTER_PATH')), 'wb')) # save the imputer


        elif self.dataset_type == "test":
            ### Clean numerical columns
            numerical_columns = X.select_dtypes(exclude=['object']).columns
            imputer = pickle.load(open(os.path.join(project_dir, os.getenv('NUMERICAL_IMPUTER_PATH')), 'rb')) # load the imputer
            X.loc[:, numerical_columns] = imputer.fit_transform(X.loc[:, numerical_columns]) # impute missing values with mean

            ### Clean categorical columns
            categorical_columns = X.select_dtypes(include=['object']).columns
            categorical_columns_missing_on_purpose = [
                'Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 
                'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 
                'GarageType', 'GarageFinish', 'GarageQual','GarageCond', 
                'PoolQC', 'Fence', 'MiscFeature'
            ]
            imputer = SimpleImputer(strategy='constant', fill_value='Missing')
            X.loc[:, categorical_columns_missing_on_purpose] = imputer.fit_transform(X.loc[:, categorical_columns_missing_on_purpose]) # impute missing categorical value on purpose
            
            imputer = pickle.load(open(os.path.join(project_dir, os.getenv('CATEGORICAL_IMPUTER_PATH')), 'rb')) # load the imputer
            X.loc[:, categorical_columns] = imputer.fit_transform(X.loc[:, categorical_columns]) # impute missing categorical value not on purpose
            
        return X


    def transform_data(self, X):
        """Either training or testing data are treate the same in this part"""
        ordinal_categorical_columns ={
            "ExterQual": ["Ex", "Gd", "TA", "Fa", "Po"], 
            "ExterCond": ["Ex", "Gd", "TA", "Fa", "Po"], 
            "BsmtQual":  ["Ex", "Gd", "TA", "Fa", "Po", "Missing"],
            "BsmtCond": ["Ex", "Gd", "TA", "Fa", "Po", "Missing"],
            "BsmtExposure": ["Gd", "Av", "Mn", "No", "Missing"],
            "BsmtFinType1": ["GLQ", "ALQ", "BLQ", "Rec", "LwQ", "Unf", "Missing"],
            "BsmtFinType2": ["GLQ", "ALQ", "BLQ", "Rec", "LwQ", "Unf", "Missing"],
            "HeatingQC": ["Ex", "Gd", "TA", "Fa", "Po"], 
            "KitchenQual": ["Ex", "Gd", "TA", "Fa", "Po"], 
            "FireplaceQu": ["Ex", "Gd", "TA", "Fa", "Po", "Missing"],
            "GarageFinish": ["Fin", "RFn", "Unf", "Missing"], 
            "GarageQual": ["Ex", "Gd", "TA", "Fa", "Po", "Missing"], 
            "GarageCond": ["Ex", "Gd", "TA", "Fa", "Po", "Missing"], 
            "PoolQC": ["Ex", "Gd", "TA", "Fa", "Missing"], 
            "Fence": ["GdPrv", "MnPrv", "GdWo", "MnWw", "Missing"]
        } # gather ordinal categorical column

        one_hot_categorical_columns = {
            "MSSubClass": ["20", "30", "40", "45",	"50", "60", "70", "75", "80", "85", "90", "120", "150", "160", "180", "190"],
            "MSZoning": ["A", "C", "FV", "I", "RH", "RL", "RP", "RM"],
            "Street": ["Pave", "Grvl"],
            "Alley": ["Missing", "Grvl", "Pave"],
            "LotShape": ["Reg", "IR1", "IR2", "IR3"],
            "LandContour": ["Lvl", "Bnk", "Low", "HLS"],
            "Utilities": ["AllPub", "NoSewr", "NoSeWa", "ELO"],
            "LotConfig": ["Inside", "FR2", "Corner", "CulDSac", "FR3"],
            "LandSlope": ["Gtl", "Mod", "Sev"],
            "Neighborhood": ["CollgCr", "Veenker", "Crawfor", "NoRidge", "Mitchel", "Somerst", "NWAmes", "OldTown", "BrkSide", "Sawyer", "NridgHt", "NAmes", "SawyerW", "IDOTRR", "MeadowV", "Edwards", "Timber", "Gilbert", "StoneBr", "ClearCr", "NPkVill", "Blmngtn", "BrDale", "SWISU", "Blueste"],
            "Condition1": ["Norm", "Feedr", "PosN", "Artery", "RRAe", "RRNn", "RRAn", "PosA", "RRNe"],
            "Condition2": ["Norm", "Artery", "RRNn", "Feedr", "PosN", "PosA", "RRAn", "RRAe", "RRNe"],
            "BldgType": ["1Fam", "2fmCon", "Duplex", "TwnhsE", "Twnhs"],
            "HouseStyle": ["2Story", "1Story", "1.5Fin", "1.5Unf", "SFoyer", "SLvl", "2.5Unf", "2.5Fin"],
            "RoofStyle": ["Gable", "Hip", "Gambrel", "Mansard", "Flat", "Shed"],
            "RoofMatl": ["CompShg", "WdShngl", "Metal", "WdShake", "Membran", "Tar&Grv", "Roll", "ClyTile"],
            "Exterior1st": ["AsbShng", "AsphShn", "BrkComm", "BrkFace", "CBlock", "CemntBd", "HdBoard",	"ImStucc", "MetalSd", "Other", "Plywood", "PreCast", "Stone", "Stucco", "VinylSd", "Wd Sdng", "WdShing"],
            "Exterior2nd": ["AsbShng", "AsphShn", "BrkComm", "BrkFace", "CBlock", "CemntBd", "HdBoard",	"ImStucc", "MetalSd", "Other", "Plywood", "PreCast", "Stone", "Stucco", "VinylSd", "Wd Sdng", "WdShing"],
            "MasVnrType": ["BrkFace", "Missing", "Stone", "BrkCmn", "CBlock"],
            "Foundation": ["PConc", "CBlock", "BrkTil", "Wood", "Slab", "Stone"],
            "Heating": ["GasA", "GasW", "Grav", "Wall", "OthW", "Floor"],
            "CentralAir": ["Y", "N"],
            "Electrical": ["SBrkr", "FuseF", "FuseA", "FuseP", "Mix", "Missing"],
            "Functional": ["Typ", "Min1", "Maj1", "Min2", "Mod", "Maj2", "Sev", "Sal"],
            "GarageType": ["Attchd", "Detchd", "BuiltIn", "CarPort", "Missing", "Basment", "2Types"],
            "PavedDrive": ["Y", "N", "P"],
            "MiscFeature": ["Missing", "Shed", "Gar2", "Othr", "TenC", "Elev"],
            "SaleType": ["WD", "New", "COD", "ConLD", "ConLI", "CWD", "ConLw", "Con", "Oth", "VWD"],
            "SaleCondition": ["Normal", "Abnorml", "Partial", "AdjLand", "Alloca", "Family"]
        } # one hot encoder column
        
        ### Numerical columns transformation
        numerical_columns = X.select_dtypes(exclude=['object']).columns
        X.loc[:, numerical_columns] = np.log1p(X.loc[:, numerical_columns]) # perform log transformation
        
        ### Categorical columns transformation
        for k, v in ordinal_categorical_columns.items():
            if k in X.columns:
                ordinal_encoder = OrdinalEncoder(categories=[v]) # define ordinal encoder
                X[k] = ordinal_encoder.fit_transform(X[[k]]).astype(int) # ordinal encoding
        
        for k, v in one_hot_categorical_columns.items():
            if k in X.columns:
                one_hot_encoder = OneHotEncoder(categories=[v], sparse_output=False, handle_unknown='ignore') # one hot encoder
                X_encoded = pd.DataFrame(one_hot_encoder.fit_transform(X[[k]])) # one hot encoding
                X_encoded.columns = one_hot_encoder.get_feature_names_out([k])
                X.drop(columns=[k], inplace=True)
                X = pd.concat([X, X_encoded], axis=1)
        return X
    
    def scale_data(self, X, scaler="standard"):
        if self.dataset_type == "train":
            if scaler == "standard":
                scaler = StandardScaler()
            elif scaler == "minmax":
                scaler = MinMaxScaler()
            elif scaler == "robust":
                scaler = RobustScaler()

            X_scaled = scaler.fit_transform(X) # scale the data    
            X = pd.DataFrame(data=X_scaled, columns=X.columns) # create pandas dataframe
            pickle.dump(scaler, open(os.path.join(project_dir, os.getenv('SCALER_PATH')), 'wb')) # save the scaler

        elif self.dataset_type == "test":
            scaler = pickle.load(open(os.path.join(project_dir, os.getenv('SCALER_PATH')), 'rb'))
            X_scaled = scaler.fit_transform(X) # scale the data    
            X = pd.DataFrame(data=X_scaled, columns=X.columns) # create pandas dataframe

        return X

    def select_column_data(self, X):
        """Either training or testing data are treate the same in this part"""
        selected_numerical_columns = [
            'LotFrontage', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 
            'MasVnrArea', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', 
            '2ndFlrSF', 'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 
            'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF'
        ]
        selected_categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
        selected_columns = np.concatenate([selected_numerical_columns, selected_categorical_columns])
        return X.loc[:, selected_columns]
# # for modelling
# class ModelTrainer():
#     pass

# # for evaluation
# class ModelEvaluator():
#     pass

In [47]:
train_data_path = os.path.join(project_dir, os.getenv('RAW_FOLDER'), 'house-prices-advanced-regression-techniques', 'train.csv')
test_data_path = os.path.join(project_dir, os.getenv('RAW_FOLDER'), 'house-prices-advanced-regression-techniques', 'test.csv')

data_preprocessor = DataPreprocessor()
data_preprocessor.preprocess_data(DataLoader.load_data(train_data_path).drop(columns=['SalePrice']), "train")
data_preprocessor.preprocess_data(DataLoader.load_data(test_data_path), "test")


DataPreprocessor is succesfully instantiated


,LotFrontage,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,...,SaleType_ConLw,SaleType_Con,SaleType_Oth,SaleType_VWD,SaleCondition_Normal,SaleCondition_Abnorml,SaleCondition_Partial,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family
0,0.589174,0.644443,0.633794,0.185615,0.000000,0.741323,0.794653,0.312002,0.000000,0.312002,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.594720,0.734867,0.611077,0.135096,0.654926,0.823054,0.842638,0.467997,0.000000,0.467997,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.554385,0.644443,0.903720,0.802409,0.000000,0.804475,0.800602,0.325885,0.870383,0.548554,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.577872,0.734867,0.911149,0.802409,0.425024,0.771613,0.800350,0.325032,0.865959,0.542433,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.313321,0.882287,0.866523,0.703168,0.000000,0.672061,0.838240,0.453130,0.000000,0.453130,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,0.000000,0.537493,0.701737,0.336710,0.000000,0.000000,0.738554,0.116114,0.837250,0.390272,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1455,0.000000,0.537493,0.701737,0.336710,0.000000,0.666931,0.738554,0.116114,0.837250,0.390272,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1456,0.899695,0.644443,0.626226,0.769361,0.000000,0.857041,0.833004,0.435427,0.000000,0.435427,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1457,0.475573,0.644443,0.866523,0.703168,0.000000,0.701843,0.798567,0.343397,0.000000,0.343397,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [4]:
"""Numerical Data Preprocessing"""
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns) -> None:
        super().__init__()
        self.columns = columns
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.loc[:, self.columns]
    

class DFSimpleImputer(SimpleImputer):
    def __init__(self, **kwargs):
        super(DFSimpleImputer, self).__init__(**kwargs)
    
    def transform(self, X):
        X_imputed = super().transform(X)
        X_imputed = pd.DataFrame(X_imputed, index=X.index, columns=X.columns)
        return X_imputed

"""Categorical Data Preprocessing"""
class MissingCategoricalOnPurposeImputer(BaseEstimator, TransformerMixin):
    def __init__(self, columns) -> None:
        super().__init__()
        self.columns = columns
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        imputer = SimpleImputer(strategy='constant', fill_value='Missing')
        X[self.columns] = imputer.fit_transform(X[self.columns])
        return X

class ModifiedOrdinalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, ordinal_categorical_columns) -> None:
        super().__init__()
        self.ordinal_categorical_columns = ordinal_categorical_columns
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        for k, v in self.ordinal_categorical_columns.items():
            ordinal_encoder = OrdinalEncoder(categories=[v])
            X[k] = ordinal_encoder.fit_transform(X[[k]]).astype(int)
        return X
    
class ModifiedOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, one_hot_categorical_columns) -> None:
        super().__init__()
        self.one_hot_categorical_columns = one_hot_categorical_columns
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        for k, v in self.one_hot_categorical_columns.items():
            one_hot_encoder = OneHotEncoder(categories=[v], sparse_output=False, handle_unknown='ignore')
            X_encoded = pd.DataFrame(one_hot_encoder.fit_transform(X[[k]]))
            X_encoded.columns = one_hot_encoder.get_feature_names_out([k])
            X.drop(columns=[k], inplace=True)
            X = pd.concat([X, X_encoded], axis=1)
        return X


ordinal_categorical_columns ={
    "ExterQual": ["Ex", "Gd", "TA", "Fa", "Po"], 
    "ExterCond": ["Ex", "Gd", "TA", "Fa", "Po"], 
    "BsmtQual":  ["Ex", "Gd", "TA", "Fa", "Po", "Missing"],
    "BsmtCond": ["Ex", "Gd", "TA", "Fa", "Po", "Missing"],
    "BsmtExposure": ["Gd", "Av", "Mn", "No", "Missing"],
    "BsmtFinType1": ["GLQ", "ALQ", "BLQ", "Rec", "LwQ", "Unf", "Missing"],
    "BsmtFinType2": ["GLQ", "ALQ", "BLQ", "Rec", "LwQ", "Unf", "Missing"],
    "HeatingQC": ["Ex", "Gd", "TA", "Fa", "Po"], 
    "KitchenQual": ["Ex", "Gd", "TA", "Fa", "Po"], 
    "FireplaceQu": ["Ex", "Gd", "TA", "Fa", "Po", "Missing"],
    "GarageFinish": ["Fin", "RFn", "Unf", "Missing"], 
    "GarageQual": ["Ex", "Gd", "TA", "Fa", "Po", "Missing"], 
    "GarageCond": ["Ex", "Gd", "TA", "Fa", "Po", "Missing"], 
    "PoolQC": ["Ex", "Gd", "TA", "Fa", "Missing"], 
    "Fence": ["GdPrv", "MnPrv", "GdWo", "MnWw", "Missing"]
} # gather ordinal categorical column

one_hot_categorical_columns = {
    "MSSubClass": ["20", "30", "40", "45",	"50", "60", "70", "75", "80", "85", "90", "120", "150", "160", "180", "190"],
    "MSZoning": ["A", "C", "FV", "I", "RH", "RL", "RP", "RM"],
    "Street": ["Pave", "Grvl"],
    "Alley": ["Missing", "Grvl", "Pave"],
    "LotShape": ["Reg", "IR1", "IR2", "IR3"],
    "LandContour": ["Lvl", "Bnk", "Low", "HLS"],
    "Utilities": ["AllPub", "NoSewr", "NoSeWa", "ELO"],
    "LotConfig": ["Inside", "FR2", "Corner", "CulDSac", "FR3"],
    "LandSlope": ["Gtl", "Mod", "Sev"],
    "Neighborhood": ["CollgCr", "Veenker", "Crawfor", "NoRidge", "Mitchel", "Somerst", "NWAmes", "OldTown", "BrkSide", "Sawyer", "NridgHt", "NAmes", "SawyerW", "IDOTRR", "MeadowV", "Edwards", "Timber", "Gilbert", "StoneBr", "ClearCr", "NPkVill", "Blmngtn", "BrDale", "SWISU", "Blueste"],
    "Condition1": ["Norm", "Feedr", "PosN", "Artery", "RRAe", "RRNn", "RRAn", "PosA", "RRNe"],
    "Condition2": ["Norm", "Artery", "RRNn", "Feedr", "PosN", "PosA", "RRAn", "RRAe", "RRNe"],
    "BldgType": ["1Fam", "2fmCon", "Duplex", "TwnhsE", "Twnhs"],
    "HouseStyle": ["2Story", "1Story", "1.5Fin", "1.5Unf", "SFoyer", "SLvl", "2.5Unf", "2.5Fin"],
    "RoofStyle": ["Gable", "Hip", "Gambrel", "Mansard", "Flat", "Shed"],
    "RoofMatl": ["CompShg", "WdShngl", "Metal", "WdShake", "Membran", "Tar&Grv", "Roll", "ClyTile"],
    "Exterior1st": ["AsbShng", "AsphShn", "BrkComm", "BrkFace", "CBlock", "CemntBd", "HdBoard",	"ImStucc", "MetalSd", "Other", "Plywood", "PreCast", "Stone", "Stucco", "VinylSd", "Wd Sdng", "WdShing"],
    "Exterior2nd": ["AsbShng", "AsphShn", "BrkComm", "BrkFace", "CBlock", "CemntBd", "HdBoard",	"ImStucc", "MetalSd", "Other", "Plywood", "PreCast", "Stone", "Stucco", "VinylSd", "Wd Sdng", "WdShing"],
    "MasVnrType": ["BrkFace", "Missing", "Stone", "BrkCmn", "CBlock"],
    "Foundation": ["PConc", "CBlock", "BrkTil", "Wood", "Slab", "Stone"],
    "Heating": ["GasA", "GasW", "Grav", "Wall", "OthW", "Floor"],
    "CentralAir": ["Y", "N"],
    "Electrical": ["SBrkr", "FuseF", "FuseA", "FuseP", "Mix", "Missing"],
    "Functional": ["Typ", "Min1", "Maj1", "Min2", "Mod", "Maj2", "Sev", "Sal"],
    "GarageType": ["Attchd", "Detchd", "BuiltIn", "CarPort", "Missing", "Basment", "2Types"],
    "PavedDrive": ["Y", "N", "P"],
    "MiscFeature": ["Missing", "Shed", "Gar2", "Othr", "TenC", "Elev"],
    "SaleType": ["WD", "New", "COD", "ConLD", "ConLI", "CWD", "ConLw", "Con", "Oth", "VWD"],
    "SaleCondition": ["Normal", "Abnorml", "Partial", "AdjLand", "Alloca", "Family"]
} # one hot encoder column

# Data Modelling 

In [11]:
# pipeline=Pipeline([
#     ('features', FeatureUnion([
#         ('numerical', Pipeline([
#             ('column_selector', ColumnSelector(columns=['LotFrontage', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', 
#                     '2ndFlrSF', 'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF'])),
#             ('simple_imputer', SimpleImputer(strategy='mean')),
#             ('logger', FunctionTransformer(np.log1p))
#         ])),
#         ('categorical', Pipeline([
#             ('column_selector', ColumnSelector(columns=categorical_columns)),
#             ('missing_categorical_on_purpose_imputer', MissingCategoricalOnPurposeImputer(columns=[
#                 'Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
#                 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual','GarageCond', 'PoolQC', 'Fence', 'MiscFeature'])),
#             ('categorical_encoder', ColumnTransformer(
#                 [
#                     ('ordinal_encoder', ModifiedOrdinalEncoder(ordinal_categorical_columns), list(ordinal_categorical_columns.keys())),
#                     ('one_hot_encoder', ModifiedOneHotEncoder(one_hot_categorical_columns), list(one_hot_categorical_columns.keys()))
#                 ], remainder="passthrough"
#             )),
#         ])),
#     ])),
#     ('scale', MinMaxScaler())
# ])

# data_preprocessor = DataPreprocessor()
train_df = DataLoader.load_data(train_data_path)
test_df = DataLoader.load_data(test_data_path)

train_df.loc[:, ('MSSubClass')] = train_df.loc[:, ('MSSubClass')].astype(str)
test_df.loc[:, ('MSSubClass')] = test_df.loc[:, ('MSSubClass')].astype(str)

numerical_columns = train_df.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = train_df.select_dtypes(include=['object']).columns

# prepare data pipeline
feature_preprocessing_pipeline=Pipeline([
    ('features', FeatureUnion([
        ('numerical', Pipeline([
            ('column_selector', ColumnSelector(columns=['LotFrontage', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', 
                    '2ndFlrSF', 'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF'])),
            ('simple_imputer', SimpleImputer(strategy='mean')),
            ('logger', FunctionTransformer(np.log1p))
        ])),
        ('categorical', Pipeline([
            ('column_selector', ColumnSelector(columns=categorical_columns)),
            ('missing_categorical_on_purpose_imputer', MissingCategoricalOnPurposeImputer(columns=[
                'Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual','GarageCond', 'PoolQC', 'Fence', 'MiscFeature'])),
            ('missing_categorical_imputer', DFSimpleImputer(strategy='most_frequent')),
            ('categorical_encoder', ColumnTransformer(
                [
                    ('ordinal_encoder', ModifiedOrdinalEncoder(ordinal_categorical_columns), list(ordinal_categorical_columns.keys())),
                    ('one_hot_encoder', ModifiedOneHotEncoder(one_hot_categorical_columns), list(one_hot_categorical_columns.keys()))
                ], remainder="passthrough"
            )),
        ])),
    ])),
    ('scale', MinMaxScaler())
])

# train_data_path = os.path.join(project_dir, os.getenv('RAW_FOLDER'), 'house-prices-advanced-regression-techniques', 'train.csv')
# test_data_path = os.path.join(project_dir, os.getenv('RAW_FOLDER'), 'house-prices-advanced-regression-techniques', 'test.csv')



# preprocessed_train_df = data_preprocessor.preprocess_data(train_df.drop(columns=['SalePrice']), "train")
# preprocessed_test_df = data_preprocessor.preprocess_data(test_df, "test")


# prepare X and y
X_train, y_train = train_df.drop(columns=['SalePrice']), train_df['SalePrice'].to_numpy()
X_test = test_df

date = datetime.datetime.now().strftime('%Y%m%d')

models = [
    LinearRegression(), Ridge(), Lasso(), 
    SVR(), DecisionTreeRegressor(), RandomForestRegressor(), 
    AdaBoostRegressor(), GradientBoostingRegressor(), ExtraTreesRegressor(), 
    XGBRegressor(), LGBMRegressor()
]
model_names = [model.__class__.__name__ for model in models]
rmse = []
for model in models:
    # train model
    X_train_preprocessed = feature_preprocessing_pipeline.fit_transform(X_train)
    # X_train_preprocessed = X_train
    # y_train_preprocessed, scaler = target_preprocessing_pipeline(y_train)
    model.fit(X_train_preprocessed, y_train.ravel())

    # calculate rmse
    y_pred = model.predict(X_train_preprocessed)
    rmse.append(np.sqrt(mean_squared_error(y_train, y_pred)))

    # predict
    X_test_preprocessed = feature_preprocessing_pipeline.transform(X_test)
    # X_test_preprocessed = X_test
    y_pred = model.predict(X_test_preprocessed)

    # save model
    pickle.dump(model, open(os.path.join(project_dir, os.getenv('MODELS_FOLDER'), f'{date}_{model.__class__.__name__.lower()}_model.pkl'), 'wb'))

    # save to submit
    pd.DataFrame({
        'Id': test_df['Id'],
        'SalePrice': y_pred.ravel()
    }).to_csv(os.path.join(project_dir, os.getenv('PROCESSED_FOLDER'), f'{date}_submission_{model.__class__.__name__.lower()}.csv'), index=False) # save

pd.DataFrame({
    'model': model_names,
    'rmse': rmse
})

c:\Users\USER\Documents\GitHub\house-prices-advanced-regression-techniques\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.920e+10, tolerance: 9.208e+08
  model = cd_fast.enet_coordinate_descent(


In [89]:
train_df = DataLoader.load_data(train_data_path)
test_df = DataLoader.load_data(test_data_path)

# train_df.loc[:, ('MSSubClass')] = train_df.loc[:, ('MSSubClass')].astype(str, inplace=True)
# train_df.info()
train_df.loc[:, ('MSSubClass')]  = train_df.loc[:, ('MSSubClass')].map(str)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   object 
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

The best model according to submission is the Random Forest Model with score of 0.1462. There are some things that we can improve such as:
1. Handling multicollinearity
2. Hyperparameter Tuning

# Evaluation
1. We still exclude handling so many zeros value on selected numerical column, and multicollinearity as well.

# Next Step

# Feature Engineering

In [107]:
x = np.log(10)
y = np.log(11)
np.exp(x*y)

249.97424928733278